<a href="https://colab.research.google.com/github/DanielUnda/Estadisticos-con-sql/blob/main/Copia_de_Netflix_Database.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install pandas DuckDB ipython-sql SQLAlchemy duckdb-engine

import duckdb
import pandas as pd
import sqlalchemy
import pyarrow as pa
from pyarrow import csv
import pyarrow.dataset as ds

%load_ext sql

%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:///:memory:

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
The sql extension is already loaded. To reload it, use:
  %reload_ext sql


/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: UserWarning: Config option `displaycon` not recognized by `SqlMagic`.  Did you mean `displaylimit`?
  exec(code_obj, self.user_global_ns, self.user_ns)


'Connected: @:memory:'

In [5]:
!pwd

/content


## Get Netflix Excel File from Tableau/Save it as CSV

In [6]:
import pandas as pd

xls = pd.ExcelFile('https://public.tableau.com/app/sample-data/netflix_titles.xlsx')
data = pd.read_excel(xls,sheet_name=None)
# print(data)

# Write the dataframe object
# into csv file
tables = []
for sheet_name, df in data.items():
  if 'cast' in df.columns:
    df.rename(columns = {'cast':'cast_member'}, inplace = True)
  df.to_csv(f'{sheet_name}.csv', index=False, header=True , encoding='utf-8')
  tables.append(sheet_name)


##Import CSV as Arrow table

In [7]:
def load_data_duckdb(csv_file,column_array=None):
    data = csv.read_csv(csv_file, convert_options=csv.ConvertOptions(
        include_columns=column_array,
        timestamp_parsers=['%m/%d/%Y %H:%M:%S']
    ))

    # DuckDB can query Arrow tables, so we'll just return the table and a connection for flexible querying
    return data, duckdb.connect(":memory:")

for table in tables:
  df_name = table + '.csv'
  print(df_name)
  globals()[f"{table}"], con = load_data_duckdb(df_name)

netflix_titles[:5]

netflix_titles.csv
netflix_titles_directors.csv
netflix_titles_countries.csv
netflix_titles_cast.csv
netflix_titles_category.csv


pyarrow.Table
duration_minutes: string
duration_seasons: string
type: string
title: string
date_added: string
release_year: double
rating: string
description: string
show_id: double
----
duration_minutes: [["90","94","","","99"]]
duration_seasons: [["","","1","1",""]]
type: [["Movie","Movie","TV Show","TV Show","Movie"]]
title: [["Norm of the North: King Sized Adventure","Jandino: Whatever it Takes","Transformers Prime","Transformers: Robots in Disguise","#realityhigh"]]
date_added: [["2019-09-09 00:00:00","2016-09-09 00:00:00","2018-09-08 00:00:00","2018-09-08 00:00:00","2017-09-08 00:00:00"]]
release_year: [[2019,2016,2013,2016,2017]]
rating: [["TV-PG","TV-MA","TV-Y7-FV","TV-Y7","TV-14"]]
description: [["Before planning an awesome wedding for his grandfather, a polar bear king must take back a stolen artifact from an evil archaeologist first.","Jandino Asporaat riffs on the challenges of raising kids and serenades the audience with a rousing rendition of "Sex on Fire" in his comedy s

In [8]:
%sql SELECT duration_minutes, duration_seasons, type FROM netflix_titles LIMIT 5;

 * duckdb:///:memory:


,duration_minutes,duration_seasons,type
0,90,,Movie
1,94,,Movie
2,,1,TV Show
3,,1,TV Show
4,99,,Movie


In [9]:
%sql SELECT director, show_id FROM netflix_titles_directors LIMIT 5;

 * duckdb:///:memory:


,director,show_id
0,Richard Finn,81145628
1,Fernando Lebrija,80125979
2,Gabe Ibáñez,70304989
3,Rodrigo Toro,80164077
4,Henrik Ruben Genz,70304990


In [10]:
%sql SELECT country, show_id FROM netflix_titles_countries LIMIT 5;

 * duckdb:///:memory:


,country,show_id
0,Germany,80016401
1,South Africa,80182274
2,United States,80182274
3,United States,81145628
4,United Kingdom,80117401


In [11]:
%sql SELECT cast_member, show_id FROM netflix_titles_cast LIMIT 5;

 * duckdb:///:memory:


,cast_member,show_id
0,Alan Marriott,81145628
1,Jandino Asporaat,80117401
2,Peter Cullen,70234439
3,Will Friedle,80058654
4,Nesta Cooper,80125979


In [12]:
%sql SELECT listed_in, show_id FROM netflix_titles_category LIMIT 5;

 * duckdb:///:memory:


,listed_in,show_id
0,Children & Family Movies,81145628
1,Stand-Up Comedy,80117401
2,Kids' TV,70234439
3,Kids' TV,80058654
4,Comedies,80125979


In [13]:
%sql SELECT typeof(duration_minutes) FROM netflix_titles;

 * duckdb:///:memory:


,typeof(duration_minutes)
0,VARCHAR
1,VARCHAR
2,VARCHAR
3,VARCHAR
4,VARCHAR
...,...
6231,VARCHAR
6232,VARCHAR
6233,VARCHAR
6234,VARCHAR


In [23]:
%sql SELECT AVG(TRY_CAST(duration_minutes AS INTEGER)) FROM netflix_titles; 

 * duckdb:///:memory:


,avg(TRY_CAST(duration_minutes AS INTEGER))
0,99.100821


In [25]:
%sql SELECT PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY TRY_CAST(duration_minutes AS INTEGER)) AS median FROM netflix_titles; 

 * duckdb:///:memory:


,median
0,98.0


In [29]:
%sql SELECT  MIN(TRY_CAST(duration_minutes AS INTEGER)) AS min  FROM netflix_titles; 

 * duckdb:///:memory:


,min
0,3


In [28]:
%sql SELECT  MAX(TRY_CAST(duration_minutes AS INTEGER)) AS max  FROM netflix_titles; 

 * duckdb:///:memory:


,max
0,312


In [30]:
%sql SELECT  MAX(TRY_CAST(duration_minutes AS INTEGER)) - MIN(TRY_CAST(duration_minutes AS INTEGER)) AS range FROM netflix_titles;

 * duckdb:///:memory:


,range
0,309


In [31]:
%sql SELECT ROUND(STDDEV(TRY_CAST(duration_minutes AS INTEGER)), 2) AS standard_deviation FROM netflix_titles; 

 * duckdb:///:memory:


,standard_deviation
0,28.07


In [32]:
%sql SELECT ROUND(SQRT(VARIANCE(TRY_CAST(duration_minutes AS INTEGER))), 2) AS stddev_using_variance FROM netflix_titles; 

 * duckdb:///:memory:


,stddev_using_variance
0,28.07


In [33]:
%sql SELECT  ROUND(VARIANCE(TRY_CAST(duration_minutes AS INTEGER)), 2) AS variance FROM netflix_titles; 

 * duckdb:///:memory:


,variance
0,788.2


In [34]:
%sql SELECT  ROUND(POWER(STDDEV(TRY_CAST(duration_minutes AS INTEGER)), 2), 2) AS variance_using_stddev FROM netflix_titles;

 * duckdb:///:memory:


,variance_using_stddev
0,788.2


In [35]:
%sql SELECT PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY TRY_CAST(duration_minutes AS INTEGER)) AS q1 FROM netflix_titles; 

 * duckdb:///:memory:


,q1
0,86.0


In [36]:
%sql SELECT PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY TRY_CAST(duration_minutes AS INTEGER)) AS q3 FROM netflix_titles; 

 * duckdb:///:memory:


,q3
0,115.0


In [37]:
%sql SELECT PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY TRY_CAST(duration_minutes AS INTEGER)) - PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY TRY_CAST(duration_minutes AS INTEGER)) AS iqr FROM netflix_titles; 

 * duckdb:///:memory:


,iqr
0,29.0


In [39]:
%%sql 
WITH mean_median_sd AS 

( 

SELECT  

AVG(TRY_CAST(duration_minutes AS INTEGER)) AS mean, 

PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY TRY_CAST(duration_minutes AS INTEGER)) AS median, 

STDDEV(TRY_CAST(duration_minutes AS INTEGER)) AS stddev 

FROM netflix_titles 

) 

SELECT  

ROUND(3 * (mean - median)::NUMERIC / stddev, 2) AS skewness 

FROM mean_median_sd; 

 * duckdb:///:memory:


,skewness
0,0.12


In [40]:
%%sql
WITH RECURSIVE summary_stats AS 

( 

SELECT  

ROUND(AVG(TRY_CAST(duration_minutes AS INTEGER)), 2) AS mean, 

PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY TRY_CAST(duration_minutes AS INTEGER)) AS median, 

MIN(TRY_CAST(duration_minutes AS INTEGER)) AS min, 

MAX(TRY_CAST(duration_minutes AS INTEGER)) AS max, 

MAX(TRY_CAST(duration_minutes AS INTEGER)) - MIN(TRY_CAST(duration_minutes AS INTEGER)) AS range, 

ROUND(STDDEV(TRY_CAST(duration_minutes AS INTEGER)), 2) AS standard_deviation, 

ROUND(VARIANCE(TRY_CAST(duration_minutes AS INTEGER)), 2) AS variance, 

PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY TRY_CAST(duration_minutes AS INTEGER)) AS q1, 

PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY TRY_CAST(duration_minutes AS INTEGER)) AS q3 

FROM netflix_titles 

),row_summary_stats AS 

( 

SELECT  

1 AS sno,  

'mean' AS statistic,  

mean AS value  

FROM summary_stats 

UNION 

SELECT  

2,  

'median',  

median  

FROM summary_stats 

UNION 

SELECT  

3,  

'minimum',  

min  

FROM summary_stats 

UNION 

SELECT  

4,  

'maximum',  

max  

FROM summary_stats 

UNION 

SELECT  

5,  

'range',  

range  

FROM summary_stats 

UNION 

SELECT  

6,  

'standard deviation',  

standard_deviation  

FROM summary_stats 

UNION 

SELECT  

7,  

'variance',  

variance  

FROM summary_stats 

UNION 

SELECT  

9,  

'Q1',  

q1  

FROM summary_stats 

UNION 

SELECT  

10,  

'Q3',  

q3  

FROM summary_stats 

UNION 

SELECT  

11,  

'IQR',  

(q3 - q1)  

FROM summary_stats 

UNION 

SELECT  

12,  

'skewness',  

ROUND(3 * (mean - median)::NUMERIC / standard_deviation, 2) AS skewness  

FROM summary_stats 

) 

SELECT *  

FROM row_summary_stats 

ORDER BY sno; 

 * duckdb:///:memory:


,sno,statistic,value
0,1,mean,99.10
1,2,median,98.00
2,3,minimum,3.00
3,4,maximum,312.00
4,5,range,309.00
5,6,standard deviation,28.07
6,7,variance,788.20
7,9,Q1,86.00
8,10,Q3,115.00
9,11,IQR,29.00


In [41]:
%sql SELECT  MODE() WITHIN GROUP (ORDER BY rating) AS mode FROM netflix_titles; 

 * duckdb:///:memory:


,mode
0,TV-MA


In [42]:
%%sql 
SELECT  

COUNT(DISTINCT rating) AS cardinality 

FROM netflix_titles; 

 * duckdb:///:memory:


,cardinality
0,15


In [43]:
%%sql 
WITH total_count AS 

( 

SELECT 

COUNT(rating) AS total_cnt 

FROM netflix_titles 

) 

SELECT  

rating, 

COUNT(rating) AS frequency, 

ROUND(COUNT(rating)::NUMERIC / (SELECT total_cnt FROM total_count), 4) AS relative_frequency 

FROM netflix_titles 

GROUP BY rating 

ORDER BY frequency DESC; 

 * duckdb:///:memory:


,rating,frequency,relative_frequency
0,TV-MA,2027,0.3250
1,TV-14,1698,0.2723
2,TV-PG,700,0.1123
3,R,508,0.0815
4,PG-13,286,0.0459
5,NR,218,0.0350
6,PG,184,0.0295
7,TV-Y7,169,0.0271
8,TV-G,149,0.0239
9,TV-Y,143,0.0229


In [44]:
%%sql 
SELECT  

rating, 

COUNT(rating) AS frequency, 

ROUND(COUNT(rating)::NUMERIC / SUM(COUNT(rating)) OVER(), 4) AS relative_frequency 

FROM netflix_titles 

GROUP BY rating 

ORDER BY frequency DESC; 

 * duckdb:///:memory:


,rating,frequency,relative_frequency
0,TV-MA,2027,0.3250
1,TV-14,1698,0.2723
2,TV-PG,700,0.1123
3,R,508,0.0815
4,PG-13,286,0.0459
5,NR,218,0.0350
6,PG,184,0.0295
7,TV-Y7,169,0.0271
8,TV-G,149,0.0239
9,TV-Y,143,0.0229
